# Setup

In [3]:
import pandas as pd
import os

import torch
from model import VAE
from utilities import load_checkpoint

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


# Load data

In [4]:
data_path = '../data/Linnarsson/midbrain_scanorama_expr.csv'
data = pd.read_csv(data_path,index_col=0).T
data

10X89_7:CTAATGGCACAACGCC  10X89_7:GCACTCTAGCAACGGT  \
TSPAN6                      -0.321850                 -0.645469   
TNMD                        -0.151038                 -0.169127   
FIRRM                       -0.494346                 -0.504340   
CFH                         -0.021953                 -0.022282   
CFTR                         0.606878                  0.605988   
...                               ...                       ...   
IQCJ-SCHIP1                  0.419309                 -0.483691   
CFAP97D2                    -0.188053                 -0.196060   
CCDC195                     -0.091549                 -0.019435   
EXOC3L2                     -0.077684                 -0.100132   
C1orf202                    -0.215652                 -0.200634   

             10X89_7:TCTCATAAGCAAATCA  10X89_7:AGTCTTTCATAGACTC  \
TSPAN6                       1.063793                  0.299839   
TNMD                        -0.083728                  0.224539   
FIRRM                       -0.515685                 -0.528414   
CFH                         -0.010128                  0.013671   
CFTR                         0.461923                  0.300879   
...                               ...                       ...   
IQCJ-SCHIP1                 -0.116496                 -0.380577   
CFAP97D2                    -0.122790                 -0.042764   
CCDC195                     -0.064358                  0.215683   
EXOC3L2                     -0.029124                 -0.087574   
C1orf202                    -0.172734                 -0.114038   

             10X89_7:AGTGTCATCAGTTCGA  10X89_7:CTCAGAACAGTAAGCG  \
TSPAN6                      -0.583755                  0.167891   
TNMD                         0.009644                 -0.016382   
FIRRM                       -0.520244                 -0.513360   
CFH                          0.012908                  0.009769   
CFTR                         0.322140                  0.397768   
...                               ...                       ...   
IQCJ-SCHIP1                  1.646913                 -0.300425   
CFAP97D2                    -0.081425                 -0.081583   
CCDC195                      0.035183                  0.053852   
EXOC3L2                     -0.050514                 -0.038701   
C1orf202                    -0.151084                 -0.160321   

             10X89_7:GATGAGGCAGCTGTGC  10X89_7:GCACTCTAGTCGTACT  \
TSPAN6                      -0.172803                  0.406667   
TNMD                        -0.015771                 -0.046833   
FIRRM                       -0.527087                 -0.515166   
CFH                          0.010293                  0.002605   
CFTR                         0.356800                  0.438313   
...                               ...                       ...   
IQCJ-SCHIP1                  0.769703                  0.113653   
CFAP97D2                    -0.112549                 -0.106098   
CCDC195                      0.006712                  0.052398   
EXOC3L2                     -0.051609                 -0.032877   
C1orf202                    -0.151925                 -0.164951   

             10X89_7:AGGTCATAGATATGGT  10X89_7:AAAGCAACATCCGCGA  ...  \
TSPAN6                       0.405796                  2.111291  ...   
TNMD                         0.079503                  0.417034  ...   
FIRRM                       -0.549209                 -0.606225  ...   
CFH                          0.023690                  0.012806  ...   
CFTR                         0.132488                  0.577829  ...   
...                               ...                       ...  ...   
IQCJ-SCHIP1                 -0.476560                 -0.305104  ...   
CFAP97D2                    -0.081045                 -0.008801  ...   
CCDC195                      0.140365                  0.170752  ...   
EXOC3L2                     -0.039087                  0.018532  ...   
C

In [5]:
data_path = '../data/Linnarsson/midbrain_meta.csv'
meta = pd.read_csv(data_path,index_col=0)
meta

CellClass  CellCycleFraction  DoubletFlag  \
10X298_6:AACGAAAGTATCGAGG   b'Neuroblast'           0.004950            0   
10X298_6:AGGATAAAGAGAGAAC   b'Neuroblast'           0.001761            0   
10X298_6:CACATGACAGAGAATT   b'Neuroblast'           0.002536            0   
10X298_6:CATGAGTCAAGCGCAA   b'Neuroblast'           0.000873            0   
10X298_6:CTAGGTAGTCGTTCAA   b'Neuroblast'           0.003597            0   
...                                   ...                ...          ...   
10X126_3:GCTCTGTAGCCACTAT  b'Radial glia'           0.003527            0   
10X126_3:GGAACTTAGTGAACGC  b'Radial glia'           0.026007            0   
10X126_3:GTGCATATCGTTACGA  b'Radial glia'           0.003635            0   
10X126_3:TCTGAGATCAAACCGT  b'Radial glia'           0.002854            0   
10X126_3:TGACTAGTCGGTCTAA  b'Radial glia'           0.004229            0   

                           DoubletScore       Region  Subdivision  \
10X298_6:AACGAAAGTATCGAGG      0.060000  b'Midbrain'  b'Midbrain'   
10X298_6:AGGATAAAGAGAGAAC      0.270000  b'Midbrain'  b'Midbrain'   
10X298_6:CACATGACAGAGAATT      0.120000  b'Midbrain'  b'Midbrain'   
10X298_6:CATGAGTCAAGCGCAA      0.030000  b'Midbrain'  b'Midbrain'   
10X298_6:CTAGGTAGTCGTTCAA      0.140000  b'Midbrain'  b'Midbrain'   
...                                 ...          ...          ...   
10X126_3:GCTCTGTAGCCACTAT      0.038462  b'Midbrain'  b'Midbrain'   
10X126_3:GGAACTTAGTGAACGC      0.076923  b'Midbrain'  b'Midbrain'   
10X126_3:GTGCATATCGTTACGA      0.076923  b'Midbrain'  b'Midbrain'   
10X126_3:TCTGAGATCAAACCGT      0.115385  b'Midbrain'  b'Midbrain'   
10X126_3:TGACTAGTCGGTCTAA      0.019231  b'Midbrain'  b'Midbrain'   

                             Subregion            Tissue  TopLevelCluster  \
10X298_6:AACGAAAGTATCGAGG  b'Midbrain'       b'Midbrain'               13   
10X298_6:AGGATAAAGAGAGAAC  b'Midbrain'       b'Midbrain'               13   
10X298_6:CACATGACAGAGAATT  b'Midbrain'       b'Midbrain'               13   
10X298_6:CATGAGTCAAGCGCAA  b'Midbrain'       b'Midbrain'               13   
10X298_6:CTAGGTAGTCGTTCAA  b'Midbrain'       b'Midbrain'               13   
...                                ...               ...              ...   
10X126_3:GCTCTGTAGCCACTAT  b'Midbrain'  b'Mesencephalon'               39   
10X126_3:GGAACTTAGTGAACGC  b'Midbrain'  b'Mesencephalon'               39   
10X126_3:GTGCATATCGTTACGA  b'Midbrain'  b'Mesencephalon'               39   
10X126_3:TCTGAGATCAAACCGT  b'Midbrain'  b'Mesencephalon'               39   
10X126_3:TGACTAGTCGGTCTAA  b'Midbrain'  b'Mesencephalon'               39   

                           TotalUMIs  ...    region    donor    sample  \
10X298_6:AACGAAAGTATCGAGG       2387  ...  Midbrain  XDD:400  10X298_6   
10X298_6:AGGATAAAGAGAGAAC      11719  ...  Midbrain  XDD:400  10X298_6   
10X298_6:CACATGACAGAGAATT       6260  ...  Midbrain  XDD:400  10X298_6   
10X298_6:CATGAGTCAAGCGCAA       4959  ...  Midbrain  XDD:400  10X298_6   
10X298_6:CTAGGTAGTCGTTCAA       4352  ...  Midbrain  XDD:400  10X298_6   
...                              ...  ...       ...      ...       ...   
10X126_3:GCTCTGTAGCCACTAT       4612  ...  Midbrain  BRC2114  10X126_3   
10X126_3:GGAACTTAGTGAACGC       2536  ...  Midbrain  BRC2114  10X126_3   
10X126_3:GTGCATATCGTTACGA       1968  ...  Midbrain  BRC2114  10X126_3   
10X126_3:TCTGAGATCAAACCGT       3798  ...  Midbrain  BRC2114  10X126_3   
10X126_3:TGACTAGTCGGTCTAA       2386  ...  Midbrain  BRC2114  10X126_3   

                                             cell_ID  \
10X298_6:AACGAAAGTATCGAGG  10X298_6:AACGAAAGTATCGAGG   
10X298_6:AGGATAAAGAGAGAAC  10X298_6:AGGATAAAGAGAGAAC   
10X298_6:CACATGACAGAGAATT  10X298_6:CACATGACAGAGAATT   
10X298_6:CATGAGTCAAGCGCAA  10X298_6:CATGAGTCAAGCGCAA   
10X298_6:CTAGGTAGTCGTTCAA  10X298_6:CTAGGTAGTCGTTCAA   
...                                              ...   
10X126_3:GCTCTGTAGCCACTAT  10X126_3:GCTCTGTAGCCACTAT   
10X126_3:GGAACTT

# Load model

## Load VAE

In [6]:
input_dim = data.shape[0]
latent_dim = 8

output_dir = '../out/midbrain/'
checkpoint_name = 'vae_499.pth'

In [7]:
vae = VAE(input_dim = input_dim, 
          latent_dim = latent_dim,
          encoder_dims = [1024, 512, 256, 128],
          decoder_dims = [128, 256, 512, 1024])
vae.eval()

checkpoint_path = os.path.join(output_dir, checkpoint_name)
load_checkpoint(checkpoint_path, vae, device) + 1

vae

VAE(
  (encoder): Sequential(
    (0): Linear(in_features=4561, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=256, bias=True)
    (5): ReLU()
    (6): Linear(in_features=256, out_features=128, bias=True)
    (7): ReLU()
  )
  (fc_mu): Linear(in_features=128, out_features=8, bias=True)
  (fc_logvar): Linear(in_features=128, out_features=8, bias=True)
  (decoder): Sequential(
    (0): Linear(in_features=8, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=512, bias=True)
    (5): ReLU()
    (6): Linear(in_features=512, out_features=1024, bias=True)
    (7): ReLU()
    (8): Linear(in_features=1024, out_features=4561, bias=True)
  )
)

## Load Psi

In [8]:
checkpoint_name = 'vae_519_trans_op.pth'
checkpoint_path = os.path.join(output_dir, checkpoint_name)
psi = torch.load(checkpoint_path, map_location=device)['psi']
psi

tensor([[[-1.3360e-02,  6.8907e-06,  1.0330e-05, -3.3358e-06,  6.6357e-06],
         [-3.2794e-02, -2.1349e-01,  1.3143e-05, -3.0300e-02, -4.0196e-06],
         [-9.2019e-06,  3.8490e-06,  1.4448e-05,  9.9183e-02, -2.6332e-06],
         [ 8.0509e-02, -7.4039e-02, -6.4198e-06,  8.6188e-02,  7.5482e-06],
         [ 1.0442e-05, -1.7787e-07, -1.5459e-05,  1.5509e-06,  1.4809e-05],
         [-9.0087e-06, -3.3726e-02, -4.4224e-06, -3.9463e-06,  1.4153e-02],
         [ 7.3746e-03, -1.2733e-06, -1.4189e-06,  1.5150e-01,  1.2306e-01],
         [-6.5070e-06,  7.8738e-06,  1.1574e-05, -5.9832e-06, -9.6031e-06]],

        [[ 3.1223e-06,  1.3710e-02,  2.3063e-06,  3.8543e-06, -3.1728e-02],
         [ 7.0484e-06, -2.2897e-06,  1.2156e-06, -6.0215e-07,  3.6258e-06],
         [ 1.0686e-01, -3.0815e-06,  1.9914e-06, -2.1179e-06,  3.6460e-02],
         [ 2.1061e-03, -1.0230e-05, -1.1114e-06, -4.1115e-06, -6.0061e-04],
         [-4.7957e-06, -3.5764e-06,  8.6635e-03,  2.5705e-02, -3.9178e-06],
         [

# Analysis

In [9]:
with torch.no_grad():
    z0, mu, _ = vae.Encode(torch.Tensor(data.T.values))

In [10]:
z1 = torch.einsum('mik, bk -> mbi', torch.matrix_exp(psi.permute(2,0,1)), z0)

In [11]:
velocity = z1 - z0

In [12]:
import anndata
import numpy as np
adata = anndata.AnnData(np.array(mu))
adata.obs = meta
adata

AnnData object with n_obs × n_vars = 21940 × 8
    obs: 'CellClass', 'CellCycleFraction', 'DoubletFlag', 'DoubletScore', 'Region', 'Subdivision', 'Subregion', 'Tissue', 'TopLevelCluster', 'TotalUMIs', 'organism_ontology_term_id', 'disease_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'assay_ontology_term_id', 'sex_ontology_term_id', 'development_stage_ontology_term_id', 'donor_id', 'suspension_type', 'dissection', 'total_UMIs', 'sample_id', 'cluster_id', 'class', 'region', 'donor', 'sample', 'cell_ID', 'OldCellID', 'Brain_region', 'Age', 'ct_score', 'ct_pseudotime', 'ct_num_exp_genes'

In [13]:
for i in range(psi.shape[-1]):
   z1_key = f'z1_psi{i}'
   v_key = f'v_psi{i}'
   adata.layers[z1_key] = np.array(z1[i, :, :])
   adata.layers[v_key] = np.array(velocity[i, :, :])

In [14]:
adata.obsm['X_mu'] = np.array(mu)
adata.obsm['X_z0'] = np.array(z0)

In [2]:
import scanpy as sc

In [15]:
sc.pp.neighbors(adata,use_rep='X_z0', random_state=0,key_added='z0')
sc.tl.umap(adata,neighbors_key='z0')
adata.obsm['X_umap_z0'] = adata.obsm['X_umap'].copy()
del adata.obsm['X_umap']
sc.pl.embedding(adata,color=['class','Age'],basis ='umap_z0')

OMP: Info #271: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


: 